
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# Mesure du pH d'une solution 

Les instruments de mesure de concentration gazeuses de la famille MQ  fournissent un signal analogique c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

## Utilisation de convertisseurs Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique dont le fonctionnement est identique. Ce qui les diffférencie est la grandeur privilégiée: fréquence d'échantillonnage pour le premier, précision de l'échantillonnage pour le second.

 * [ADS1015](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1015.html) Résolution de 12 bit, Fréquence d'échantillonnage Min : 128 Hz Max :3300 Hz (SPS_128 à SPS_3300)
 * [ADS1115](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1115.html) Résolution  de 16 bit, Fréquence d'échantillonage Min : 8 Hz Max : 860 Hz (SPS_8 à SPS_860)

dont le fonctionnement est totalement identique. Ce qui les diffférencie est la grandeur privilégiée, pour le premier c'est la fréquence de l'échantillonnage aux dépens de l'échantillonnage de la valeur mesurée pour le second c'est l'inverse. La totalité des commandes sont communes sauf la gamme de réglage de ces deux paramètres, hautes fréquences pour le `ADS1015`et basses fréquences pour le `ADS1115`.

Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie
Nous disposons de 3 capteurs et de 4 broches d'échantillonnage sur le convertisseur, nous ferons deux mesures directes et une mesure par différence. L'exploitation fine de ces éléments requiert le calibrage des capteurs analogiques et le choix des bons paramètres de configuration. L'exemple simple présenté ici a été réalisé en mode direct pour le capteur qualité et le capteur aclcool en mode différentiel pour le capteur méthane.

La figure ci-dessous présente le matériel acheté pour mesurer le pH (et la température)

https://www.diymore.cc/collections/ph-value-detect-sensor

![Convertisseurs vue générale](images/pHSensorDIY.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Le câblage du convertisseurs ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |


L'image suivante montre la connexion de la carte connectée au RaspberyPi et en BNC au capteur de pH

![Convertisseurs vue du câblage](images/pH_montage.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)


Importation des librairies nécessaires à l'utilisation du convertisseur. On n'importe aucune librairie propre aux capteurs de gaz car il s'agit de capteurs `analogiques` qui transmettent une variation de potentiel sur un fil série.

In [1]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

La configuration se base sur le [descriptif du composant (datasheet)](https://cdn-shop.adafruit.com/datasheets/ads1015.pdf)

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer.
 * setGain()
 * setSPS()
 * setContinuous()
 
Le paramètre SPS contrôle la fréquence d'échantillonnage, pour des mesures de teneur en gaz les basses fréquences de l'`ADS1115` sont largement suffisantes.

Pour ADS1015  :

la commande est `setSPS(rate=SPS_1600)`; les valeurs possibles étant :  SPS_128, SPS_250, SPS_490, SPS_920, SPS_1600, SPS_2400, SPS_3300.

Pour ADS1115 : 

la commande est `setSPS(rate=SPS_32)`; les valeurs possibles étant :  SPS_8, SPS_16, SPS_32, SPS_64,
  SPS_128, SPS_250, SPS_475, SPS_860. 
  
Le choix du gain est aussi un paramètre crucila les valeurs possibles sont :  `GAIN_TWOTHIRDS` = ADS1X15_PGA_6_144V, `GAIN_ONE` = ADS1X15_PGA_4_096V, `GAIN_TWO` = ADS1X15_PGA_2_048V, `GAIN_FOUR` = ADS1X15_PGA_1_024V,
  `GAIN_EIGHT` = ADS1X15_PGA_0_512V, `GAIN_SIXTEEN` = ADS1X15_PGA_0_256V. La valeur doit être choisie en fonction des propriétés du capteur.

In [2]:
sensorph = upm.ADS1015(0, 0x49)
gain = sensorph.getGain()
sps = sensorph.getSPS()
cont = sensorph.getContinuous() 
rate=sensorph.SPS_128
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))
sensorph.setContinuous(True) # Pour ne pas faire de conversion en continu
sensorph.setGain(upm.ADS1X15.GAIN_ONE) # Passage à un gain 6.144 V
sensorph.setSPS(rate) # Passage à une fréquence de 250
# Vérification
gain = sensorph.getGain()
sps = sensorph.getSPS()
cont = sensorph.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:2.2f}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :   128 Continuous :     0

Nouvelles valeurs :
Gain : 512.00  SPS :     0 Continuous :     1


## Mesure directe des variables pH et température

Les deux premières mesures sont effectués directement, la troisième est faite par différence.

In [3]:
pH = sensorph.getSample(upm.ADS1X15.SINGLE_1)
temp = sensorph.getSample(upm.ADS1X15.SINGLE_3)
diff_temp = sensorph.getSample(upm.ADS1X15.DIFF_0_3)
diff_pH = sensorph.getSample(upm.ADS1X15.DIFF_0_1)
print ("pH : {0:.10f}\t".format(pH),
       "Température : {0:.10f}\t".format(temp),
       "pH diff : {0:.7f}\t".format(diff_temp),
       "Temp diff : {0:.7f}\n".format(diff_pH))

pH : 1.5100001097	 Température : 3.6800000668	 pH diff : 2.1600001	 Temp diff : 3.7100003



# série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 20 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [5]:
for num in range(0,5):
    pH = sensorph.getSample(upm.ADS1X15.SINGLE_1)
    temp = sensorph.getSample(upm.ADS1X15.SINGLE_3)
    diff_temp = sensorph.getSample(upm.ADS1X15.DIFF_0_3)
    diff_pH = sensorph.getSample(upm.ADS1X15.DIFF_0_1)
    print ("pH : {0:.10f}\t".format(pH),
       "Température : {0:.10f}\t".format(temp),
       "pH diff : {0:.10f}\t".format(diff_temp),
       "Temp diff : {0:.10f}\t".format(diff_pH))    
    sleep(1)

pH : 2.1579999924	 Température : 3.7040002346	 pH diff : 2.1620001793	 Temp diff : 3.7020001411	
pH : 2.1620001793	 Température : 3.7200002670	 pH diff : 2.1620001793	 Temp diff : 3.7260000706	
pH : 2.1600000858	 Température : 3.7100002766	 pH diff : 2.1640000343	 Temp diff : 2.1620001793	
pH : 2.1620001793	 Température : 3.7080001831	 pH diff : 2.1640000343	 Temp diff : 3.7360000610	
pH : 2.1620001793	 Température : 3.7280001640	 pH diff : 2.1640000343	 Temp diff : 3.7040002346	
